Nama : Lopinta Sarungallo

NIM : H071191025

Soal: Pelajari Chapter 5 dari d2l.ai : Deep Learning Computation

In [ ]:
import tensorflow as tf
from tensorflow import keras

# 1. Layer dan Blocks

Layer merupakan arsitektur atau struktur yang berisi kumpulan node yang beroperasi secara dan di waktu yang bersamaan, yang mengambil informasi dari layer sebelumnya dan kemudian meneruskan informasi ke layer berikutnya.

Block atau neural network blocks merupakan konsep di mana satu atau lebih layer digambarkan / diabstraksi menjadi satu komponen (block). Salah satu manfaat utama dari abstraksi blocks adalah blocks ini bisa lagi dikombinasikan menjadi struktur yang lebih besar (biasanya secara rekursif).

Dengan menggunakan konsep blocks ini, kita tetap dapat menulis kode yang ringkas dan masih mengimplementasikan neural net yang kompleks. Untuk mengimplementasikan blocks, kita membuat fungsi atau jika ingin fleksibel, kelas turunan dari tensorflow.keras.Model yang memiliki fungsi forward propagation (fungsi call()) . 

Contoh implementasi kode


In [ ]:
# Data: digenerate secara random
X = tf.random.uniform((2, 20))

X

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.7250458 , 0.7812331 , 0.2860253 , 0.13792169, 0.7839724 ,
        0.13899052, 0.22877407, 0.5630474 , 0.5698296 , 0.29201913,
        0.8923713 , 0.11934936, 0.5118754 , 0.08690977, 0.31809747,
        0.3914007 , 0.73344004, 0.9947243 , 0.99763954, 0.07119775],
       [0.6998304 , 0.8987254 , 0.58470654, 0.87934923, 0.18865657,
        0.6503935 , 0.22828412, 0.17233586, 0.50217617, 0.5718585 ,
        0.17911935, 0.65175235, 0.19903302, 0.4745115 , 0.5310135 ,
        0.76653194, 0.12745714, 0.2762015 , 0.14449227, 0.644029  ]],
      dtype=float32)>

Misalnya model yang akan dijadikan blok merupakan model seperti ini

In [ ]:
block1 = keras.models.Sequential([
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10),
])

Bentuk bloknya

In [ ]:
class Block1(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer_list= []
        self.layer_list.append(keras.layers.Dense(256, activation=tf.nn.relu))
        self.layer_list.append(keras.layers.Dense(10))
        self.layer_list.reverse()
    
    def call(self, X):
        result_temp= X

        # Melakukan forward propagation sampai layer terakhir dalam list 
        while bool(self.layer_list):
            result_temp= self.layer_list.pop()(result_temp)

        return result_temp
        # return 1

In [ ]:
net= Block1()
net(X) # Sama saja dengan net.call(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 1.8904912e-01, -2.1566553e-02,  4.5625612e-01, -1.2845834e-01,
        -9.1123998e-02, -7.1828671e-02,  1.6553319e-01,  4.4604097e-03,
        -1.5544681e-01,  4.6881079e-05],
       [ 2.2053109e-02,  2.1610294e-02,  5.8731389e-01, -1.8332723e-01,
        -2.7495167e-01,  1.5651681e-01, -1.2013686e-01,  1.7667633e-01,
        -1.1486124e-01,  1.0844171e-01]], dtype=float32)>

Selanjutnya kita akan membuat block ke dua yang tidak mengandung layer satu per satu tapi mengandung model

In [ ]:
class Block2(keras.Model):
    def __init__(self):
        super().__init__()
        self.net= keras.Sequential()
        self.net.add(keras.layers.Dense(256, activation=tf.nn.relu))
        self.net.add(keras.layers.Dense(10))
    
    def call(self, X):
        return self.net(X)

Kemudian satukan

In [ ]:
net= keras.Sequential()

# Menambahkan layer pertama yang mana merupakan block 2
net.add(Block2())

# Menambahkan block 1
net.add(Block1())

# # Menambahkan satu lagi layer dense biasa
net.add(keras.layers.Dense(10, activation= 'sigmoid'))

# Melakukan forward propagation
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[0.49946293, 0.41505975, 0.50440824, 0.4758867 , 0.50704443,
        0.4546392 , 0.45419297, 0.5167015 , 0.49893108, 0.43906072],
       [0.4121948 , 0.48233908, 0.5370872 , 0.42183027, 0.5670335 ,
        0.53832537, 0.52566886, 0.542594  , 0.54677814, 0.4792155 ]],
      dtype=float32)>

# 2. Parameter Management

Parameter management atau manajemen parameter termasuk (Parameter dalam kasus neural network contohnya weight dan bias, fungsi loss atau learning rate merupakan hyperparameter):

Mengakses parameter untuk debugging, diagnostik, dan visualisasi.
Inisialisasi parameter.
Berbagi parameter dengan komponen model lain.



## Mengakses Parameter

Model:

In [ ]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu),
    keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.17184342],
       [0.35465258]], dtype=float32)>

Mengambil weight dan bias dari setiap layer net

In [ ]:
net.weights
# Atau net.get_weights()

[<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.8619006 , -0.02977639, -0.6566596 ,  0.509875  ],
        [ 0.05028754, -0.6336104 ,  0.44476324, -0.82829285],
        [-0.28720802,  0.8080464 ,  0.4687975 ,  0.34760433],
        [ 0.42062885, -0.48057932,  0.30534905, -0.34293258]],
       dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_10/kernel:0' shape=(4, 1) dtype=float32, numpy=
 array([[ 0.40594113],
        [-1.0012045 ],
        [ 0.9572525 ],
        [ 0.9913094 ]], dtype=float32)>,
 <tf.Variable 'dense_10/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

Untuk mendapatkan parameter pada layer yang ingin diakses, dilakukan dengan memanggil atribut .layers[index dari layer] dari model sequential. Misal layer yang ingin diakses parameternya yaitu layer indeks ke-1

net.layers[1]

Selanjutnya untuk mengakses parameter weight dan bias

In [ ]:
net.layers[1].weights

[<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.8619006 , -0.02977639, -0.6566596 ,  0.509875  ],
        [ 0.05028754, -0.6336104 ,  0.44476324, -0.82829285],
        [-0.28720802,  0.8080464 ,  0.4687975 ,  0.34760433],
        [ 0.42062885, -0.48057932,  0.30534905, -0.34293258]],
       dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

Outputnya berupa array, indeks ke-0 merupakan weights, indeks ke-1 merupakan bias

Untuk layer yang berupa block (memiliki layer lagi di dalamnya), kita tetap bisa mengakses nya seperti ini:

net.layers[0].layers[1].layers[1].weights[1]

### Inisialisasi Parameter

Secara default (untuk tensorflow), matriks weight diinisialisasikan berdasarkan range yang dikomputasi dari dimensi input dan output, sementara parameter bias mempunyai nilai default 0.

Misal menginisialisasi weight sebagai variabel random Gaussian dengan standar deviasi 0.01, dan menginisialisasi parameter bias dengan nilai 0.

In [ ]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(1)
])

net(X)
net.layers[1].weights[0], net.layers[1].weights[1]

(<tf.Variable 'dense_11/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.0004697 ,  0.01119652,  0.00130588, -0.00839333],
        [-0.01070861,  0.00124902,  0.00244164, -0.00724455],
        [-0.02358088, -0.00429182, -0.00405262,  0.00609773],
        [ 0.00687516,  0.01036162, -0.00047495, -0.01063034]],
       dtype=float32)>,
 <tf.Variable 'dense_11/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

Kita juga bisa menginisialisasikan weight yang berbeda pada setiap layer, misal weight layer dense kedua diisi dengan angka konstan 0.5.

In [ ]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=keras.initializers.Constant(0.5), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(1)
])

net(X)
net.layers[1].weights[0], net.layers[2].weights[0]

(<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.01897863, -0.02186587, -0.00879242, -0.00187525],
        [ 0.01498427,  0.00489209,  0.0017231 ,  0.00040562],
        [ 0.02421116,  0.00943782,  0.01279427,  0.00351813],
        [-0.00221438, -0.0045473 ,  0.00440709, -0.02620335]],
       dtype=float32)>,
 <tf.Variable 'dense_14/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5]], dtype=float32)>)

Kita juga bisa membuat subclass kita sendiri untuk inisialisasi weight jika cara di atas kurang flexibel, caranya yaitu kita membuat class subclass dari keras.initializers.Initializer , lalu mengimplementasikan method call(shape, dtype) yang me-return tensor (berisi weight inisial) yang isinya sesuai kemauan kita

In [ ]:
# Di sini kita membuat class untuk inisialisasi weight
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

In [ ]:
MyInit().__call__(shape=(3, 3), dtype=tf.float32)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-0.       , -0.       , -9.723315 ],
       [ 5.3541164,  6.115057 ,  0.       ],
       [ 0.       ,  8.298063 , -0.       ]], dtype=float32)>

In [ ]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    # Memasukkan kelas initializer pada parameter kernel_initializer
    keras.layers.Dense(4, activation=tf.nn.relu, kernel_initializer=MyInit()),
    keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_16/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.       , -8.799205 , -0.       , -8.715923 ],
       [-0.       ,  0.       ,  0.       , -5.374882 ],
       [ 0.       , -5.806229 , -6.8254805,  7.19158  ],
       [-5.430324 , -5.879984 ,  6.528591 ,  0.       ]], dtype=float32)>


# 3. Deferred Initialization
Inisialisasi weight pada model ditangguhkan sebelum data di-pass ke model. Ini dikarenakan model masih belum mengetahui input shape dari data, maka dari itu model menangguhkan inisialisasi parameter, konsep ini juga membuat kita tidak perlu mendefinisikan input shape dari setiap layer, yang mana pastinya sangat membantu terutama jika dataset berupa gambar (yang mana mempunyai shape yang dinamis)

Contohnya seperti kasus berikut, pertama-tama kita membuat model lalu memanggil weight nya

In [ ]:
net = keras.models.Sequential([
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10),
])
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

Kita bisa melihat weightnya masih kosong karena belum diinisialisasikan sama sekali

In [ ]:
# Feed model dengan X
net(X)

[w.shape for w in net.get_weights()], net.layers[0].get_weights()[0]

([(4, 256), (256,), (256, 10), (10,)],
 array([[-0.11825895,  0.06696358,  0.06014265, ..., -0.00091687,
          0.108962  ,  0.04111561],
        [-0.11580317,  0.07439198, -0.0238121 , ..., -0.00879824,
         -0.10376012,  0.09112103],
        [-0.12448703, -0.03093658,  0.07049052, ..., -0.06845955,
         -0.09233275, -0.07080831],
        [-0.07194716,  0.07404892, -0.06215306, ..., -0.08092736,
          0.14182144,  0.01678324]], dtype=float32))

Barulah setelah data di-pass weightnya terinisialisasi. Ini dinamakan Deferred Initialization (penangguhan inisialisasi).

# 4. Custom Layers
Pada kasus-kasus tertentu, kita berhadapan dengan masalah di mana kita membutuhkan layer yang belum pernah dibuat atau belum ada pada library, untuk mengatasi masalah tersebut, kita bisa membuat layer kustom kita sendiri. Membuat layer kustom sama saja seperti membuat blok. Terdapat 2 jenis kustom layer: layer tanpa atau dengan parameter (seperti weight atau bias)

### Layer tanpa parameter


Layer tanpa parameter merupakan layer yang tidak mempunyai weight dan bias di dalamnya (seperti layer flatten). Misal kita ingin membuat layer yang mengurangi input dengan satu

In [ ]:
class MinusOneLayer(keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, input):
        return input - tf.ones(input.shape)

In [ ]:
layer_wo_params= MinusOneLayer()
layer_wo_params(tf.constant([1.,2.,3.]))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 1., 2.], dtype=float32)>

Mengimplementasikannya pada model

In [ ]:
net = keras.Sequential([keras.layers.Dense(128), MinusOneLayer()])

### Layer dengan parameter
Layer dengan parameter mempunyai dan melibatkan weight dan bias pada operasinya. Misal kita ingin membuat layer dense yang berfungsi sama dengan layer dense pada umumnya (input * weight + bias lalu aktivasi). Walaupun kita mengatur/inisialisasi parameter semau kita, parameter juga tetap berubah saat proses back propagation

In [ ]:
class MyDense(keras.Model):
    # Di sini kita berikan dua parameter saja yaitu untuk output dan aktivasi (tentu saja kita bisa menambahkannya sesuai kebutuhan)
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation= activation

    # Pada method build kita inisialisasi weight dan bias, fungsi build hanya dijalankan satu kali
    # Jadi weight dan bias tetap akan ter-update saat back propagation
    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    # Pada fungsi call kita melakukan operasi (input * weight + bias lalu aktivasi)
    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias

        if self.activation is None:
            return linear
        
        return self.activation(linear)

In [ ]:
temp_dense= MyDense(5)
temp_dense(X)

temp_dense.get_weights()

[array([[-0.05568484,  0.00387324, -0.02981849, -0.04221389, -0.06546652],
        [ 0.04056905, -0.09633523, -0.11978207, -0.03083634,  0.02450412],
        [-0.03103748,  0.00274478,  0.03435281, -0.09713116, -0.0656438 ],
        [-0.09470006,  0.02267141, -0.01109368,  0.00492686,  0.02717558]],
       dtype=float32), array([0., 0., 0., 0., 0.], dtype=float32)]

Selanjutnya membuat model dengan 2 layer MyDense yang telah dibuat

In [ ]:
net= keras.Sequential([
    keras.layers.Flatten(),
    MyDense(128, tf.nn.relu), 
    MyDense(64, tf.nn.relu), 
    MyDense(32, tf.nn.relu), 
    MyDense(10, tf.nn.softmax)
])

net.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Kita akan mengetes model tersebut pada dataset mnist

In [ ]:
(x_train, y_train), (x_test, y_test)= keras.datasets.mnist.load_data()
x_train, x_test= x_train/255, x_test/255

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
net.fit(x_train, y_train, epochs=3, batch_size= 128)

Epoch 1/3
469/469 [==============================] - 5s 5ms/step - loss: 0.5305 - accuracy: 0.8451
Epoch 2/3
469/469 [==============================] - 2s 4ms/step - loss: 0.2045 - accuracy: 0.9411
Epoch 3/3
469/469 [==============================] - 2s 4ms/step - loss: 0.1417 - accuracy: 0.9585


In [ ]:
net.evaluate(x_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 0.1271 - accuracy: 0.9644


[0.1271296590566635, 0.9643999934196472]

# 5. File I/O

### Save / Load Tensor
Untuk tensor, kita bisa menggunakan numpy.save(namaFile.npy, tensor) untuk menyimpan tensor dan numpy.load(namaFile.npy) untuk memuatnya.

In [ ]:
save_x= tf.constant(['a', 'b', 'c'])
save_x

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'a', b'b', b'c'], dtype=object)>

Menyimpan tensor

In [ ]:
import numpy as np

np.save('save-x.npy', save_x)

Meload tensor

In [ ]:
temp_x= np.load('save-x.npy', allow_pickle=True)
temp_x

array([b'a', b'b', b'c'], dtype=object)

### Save / Load Model Parameter
Misal model kita seperti ini

In [ ]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net= MLP()

Lalu kita melakukan feed forward pada model

In [ ]:
Y1= net(X)
Y1

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.02405046,  0.13314383,  0.0316291 ,  0.07723983,  0.0941774 ,
         0.09787807, -0.09560215, -0.00592709,  0.05555975,  0.0734397 ],
       [-0.04398845,  0.09163702,  0.05900822,  0.06908892,  0.08900558,
         0.17349096,  0.01819794, -0.0093819 ,  0.00583143,  0.02785756]],
      dtype=float32)>

Sekarang kita akan menyimpan parameter model dengan fungsi save_weights('nama_weight.params')

In [ ]:
net.save_weights('mlp.params')

Sekarang kita akan membuat model dengan arsitektur yang sama tapi tidak dilakukan feed forward, sebaliknya parameternya diambil/diload dari file parameter yang telah di-save sebelumnya

In [ ]:
net_2= MLP()
net_2.load_weights('mlp.params')

# Feed forward
Y2= net_2(X)

Sekarang kita akan membandingkan variabel Y1 dengan Y2

In [ ]:
Y1==Y2

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

Hasilnya adalah sama

# 6. GPUs
Mengecek ketersediaan GPU

In [ ]:
!nvidia-smi

Sat Apr 23 12:53:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    71W / 149W |    707MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Kita dapat menentukan device, seperti CPU dan GPU, untuk penyimpanan dan komputasi. Secara default, tensor dibuat di memori utama dan menggunakan CPU untuk komputasi.

In [ ]:
tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f88eb4b3b40>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f88eb4b3a50>)

Kita bisa menjalankan query untuk mendapatkan berapa banyak GPU dalam sistem

In [ ]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

### Tensor dan GPU

Secara default, tensor dibuat dalam CPU, untuk melihat di device mana suatu tensor berada kita bisa menjalankan fungsi seperti ini

In [ ]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:GPU:0'

Outputnya mengatakan GPU karena saya menggunakan google colabs dan telah mengatur runtime pada saat penulisan kode, namun secara default, kalau tidak didefinisikan, akan mengeluarkan output CPU

### Menyetel penggunaan GPU

Misal kita ingin menggunakan GPU pertama atau GPU:0

In [ ]:
tf.device('GPU:0')

In [ ]:
with tf.device('GPU:0'):
    test = tf.ones((2, 3))

test.device

'/job:localhost/replica:0/task:0/device:GPU:0'

Hati-hati saat pemilihan GPU terutama jika kita memiliki lebih dari satu GPU dalam satu komputer, karena akan terjadi error jika kita melakukan operasi yang melibatkan 2 tensor yang berada di GPU yang berbeda